# Imports

In [ ]:
import os
import pandas as pd
from typing import Optional
import waffles.input_output.raw_hdf5_reader as reader
import waffles.np04_analysis.led_calibration.utils as led_utils

# Tools definitions

In [ ]:
def save_run_to_pickled_WaveformSet(
    run: int,
    saving_folderpath: str,
    average_wfs_per_channel: int = 4000,
    channels: Optional[dict] = None,
    channels_no: int = 40,
    rucio_filepaths_folderpath: str = "/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-II/PDS_Commissioning/waffles/1_rucio_paths/",
    read_full_streaming_data: bool = False,
    self_trigger_endpoints: list[int] = [109, 111, 112, 113],
    full_streaming_endpoints: list[int] = [104, 105, 107],
    subsample_seed: int = 3,
    verbose: bool = True
    ) -> None:
    """This function gets the following positional arguments:

    - run (int): Number of the run whose data we want to convert
    to a pickle'd WaveformSet.
    - saving_folderpath (str): Path to the folder where to save
    the pickle'd WaveformSet(s).

    This function gets the following keyword arguments:

    - average_wfs_per_channel (int): Assuming that the read data
    is homogeneously distributed along the detector channels,
    the pickle'd WaveformSet object(s) will contain, on average,
    average_wfs_per_channel Waveform objects per detector
    channel.
    - channels (dictionary): A dictionary with the channels
    to be read for this run. The keys of the dictionary
    are the endpoint values, while the values are the
    channel numbers. If this parameter is defined, then
    the channels_no parameter is ignored, and the channels
    number is computed as the number of different channels
    in this dictionary. However, note that prior to this
    counting, the endpoints in this dictionary are filtered
    according to the following three parameters:
    read_full_streaming_data, self_trigger_endpoints and
    full_streaming_endpoints. The filtered channels parameter
    is given to the 'ch' parameter of the
    WaveformSet_from_hdf5_file() function. Check such function
    docstring for more information.
    - channels_no (int): Number of channels in the detector.
    This parameter makes a difference only if the 'channels'
    parameter is not defined.
    - rucio_filepaths_folderpath (str): Path to the folder
    where the files with the rucio filepaths are stored.
    The file which contains the rucio filepaths for a
    given run number, <run>, is assumed to be called 
    '0<run>.txt',
    - read_full_streaming_data (bool): Whether to read the
    full-streaming data of the self-trigger data.
    - self_trigger_endpoints (resp. full_streaming_endpoints)
    (list of integers): The list of endpoints which contain
    self-triggered (resp. full-streaming) channels. This
    parameter makes a difference only if the 'channels'
    parameter is defined. Indeed, the
    WaveformSet_from_hdf5_file() function knows per se which
    endpoints/channels are self-triggered or full-streamed.
    However, if the 'channels' parameter is defined, we need
    to know this information prior to calling
    WaveformSet_from_hdf5_file() so that we can compute the
    actual number of channels which we are targeting, and
    so, the number of waveforms that we need to read in
    order to get an average of average_wfs_per_channel
    waveforms per channel.
    - subsample_seed (int): The seed for the subsample
    parameter. This parameter is decreased unit by unit
    until the number of Waveform objects in the resulting
    WaveformSet object reaches
    average_wfs_per_channel * number_of_channels, where
    number_of_channels is the number of different channels
    read for this run. This parameter is given to the
    'subsample' parameter of the
    WaveformSet_from_hdf5_file() function. Check such
    function docstring for more information.
    - verbose (bool): Whether to print functioning-related
    messages.
    
    This function looks for a file called '0<run>.txt' within
    the folder whose path is given by 
    rucio_filepaths_folderpath. Such file is assumed to be a
    text file with a list of filepaths. Parsing such file is
    delegated to the get_filepaths_from_rucio() function of
    'raw_hdf5_reader.py' module. If it is found, then 
    it starts reading WaveformSet(s), one per filepath, until
    the total number of read waveforms has reached
    average_wfs_per_channel * number_of_channels waveforms,
    where number_of_channels is the number of different
    channels read for this run. Reading the WaveformSet(s)
    is delegated to the WaveformSet_from_hdf5_file() function
    of the 'raw_hdf5_reader.py' module. The read WaveformSet(s)
    are pickle'd to files which are saved in the folder pointed 
    to by saving_folderpath. The WaveformSet coming from 
    the i-th filepath is saved to the file named
    'run_<run>_chunk_<i>_FS.pkl' if read_full_streaming_data is
    True, and 'run_<run>_chunk_<i>_ST.pkl' otherwise.
    """

    if channels is None:
        fChannelsAreDefined = False
        number_of_channels = channels_no

    else:
        # In this case, compute the number of different
        # channels making sure that there are no duplicates
        fChannelsAreDefined = True
        number_of_channels = 0
        channels_ = {}

        # Filter the given 'channels' parameter to:
        #   - erase endpoints with no targeted channels
        #   - erase endpoints whose type (full-streaming
        #     or self-trigger) do not match the 
        #     read_full_streaming_data parameter
        #   - erase duplicated channels
        for endpoint in channels.keys():
            if len(channels[endpoint]) > 0:
                if read_full_streaming_data:
                    if endpoint in full_streaming_endpoints:
                        channels_[endpoint] = list(set(channels[endpoint]))
                        number_of_channels += len(channels_[endpoint])
                else:
                    if endpoint in self_trigger_endpoints:
                        channels_[endpoint] = list(set(channels[endpoint]))
                        number_of_channels += len(channels_[endpoint])

        if len(channels_) == 0:
            raise Exception(
                "In function save_run_to_pickled_WaveformSet(): "
                f"After filtering the given channels parameter ({channels}) "
                "no channels were left. Make sure that the type of data "
                "which you are requiring (read_full_streaming_data="
                f"{read_full_streaming_data}) matches the specified run "
                f"({run})."
            )

    aux = os.path.join(
        rucio_filepaths_folderpath,
        f"0{run}.txt"
    )

    try:
        rucio_filepaths = reader.get_filepaths_from_rucio(aux)
    # Happens if there are no rucio filepaths for this run in rucio_filepaths_folderpath
    except Exception:
        print(
            f"In function save_run_to_pickled_WaveformSet(): The rucio "
            f"paths for run {run} were not found. Ending execution of "
            f"save_run_to_pickled_WaveformSet({run}, ...)."
        )
        return

    chunks_no = len(rucio_filepaths)
    if verbose:
        print(
            "In function save_run_to_pickled_WaveformSet(): "
            f"Processing run {run} - found {chunks_no} chunk(s) ..."
        )

    fGoForAnotherChunk = True
    wvfs_left_to_read_for_this_run = average_wfs_per_channel * number_of_channels
    current_chunk_iterator = 0

    while fGoForAnotherChunk:

        if verbose:
            print(
                f"\t --> Processing chunk {current_chunk_iterator+1}"
                f"/{chunks_no} ..."
            )

        subsample = subsample_seed
        fReadSameChunkAgain = True

        while fReadSameChunkAgain:

            aux_wfset = reader.WaveformSet_from_hdf5_file(
                rucio_filepaths[current_chunk_iterator],
                read_full_streaming_data=read_full_streaming_data,
                subsample=subsample,
                # WaveformSet_from_hdf5_file apparently subsamples from
                # the [0, wvfm_count] range. Therefore, if we set
                # wvfm_count to wvfs_left_to_read_for_this_run we
                # will get, at most, wvfs_left_to_read_for_this_run/subsample
                wvfm_count=wvfs_left_to_read_for_this_run*subsample,
                # WaveformSet_from_hdf5_file ignores the 'ch' parameter
                # if it is an empty dictionary
                ch=channels_ if fChannelsAreDefined else {}
            )
            
            # In this case, we already have what we need for this run
            if len(aux_wfset.waveforms) == wvfs_left_to_read_for_this_run:
                fReadSameChunkAgain = False
                fGoForAnotherChunk = False

                aux = f"run_{run}_chunk_{current_chunk_iterator}_awpc_"+\
                    f"{round(len(aux_wfset.waveforms)/number_of_channels)}"
                aux_saving_filename = f"{aux}_FS.pkl" if read_full_streaming_data else f"{aux}_ST.pkl"

                if verbose:
                    print(
                        "\t\t --> Got enough waveforms ("
                        f"{len(aux_wfset.waveforms)}) from chunk "
                        f"{current_chunk_iterator+1}/{chunks_no} "
                        f"of run {run}"
                    )
                    print(
                        "\t --> Now saving it to a pickle file in "
                        f"{os.path.join(saving_folderpath, aux_saving_filename)} ..."
                    )

                led_utils.dump_object_to_pickle(
                    aux_wfset,
                    saving_folderpath,
                    aux_saving_filename,
                    verbose=verbose
                )
                
                if verbose:
                    print(
                        "In function save_run_to_pickled_WaveformSet(): "
                        f"Successfully saved WaveformSet of run {run}"
                    )

            # In this case, we need more waveforms for this run
            elif len(aux_wfset.waveforms) < wvfs_left_to_read_for_this_run:

                if verbose:
                    print(
                        f"\t\t --> Didn't get enough waveforms from chunk "
                        f"{current_chunk_iterator+1}/{chunks_no} of run {run}"
                    )
                    print(
                        f"\t\t --> Expected {wvfs_left_to_read_for_this_run}, "
                        f"but only read {len(aux_wfset.waveforms)}"
                    )

                # In this case, try to read the same file but with a finer subsampling
                if subsample > 1:
                    # fReadSameChunkAgain is True by default
                    previous_subsample = subsample
                    subsample = led_utils.next_subsample(
                        subsample, # current_subsample
                        len(aux_wfset.waveforms), # read_quantity
                        wvfs_left_to_read_for_this_run # required_quantity
                    )

                    if verbose:
                        print(
                            f"\t\t --> Switching 'subsample' from {previous_subsample} "
                            f"to {subsample} and reading it again ..."
                        )

                # In this case, we read every waveform from this chunk, but we still
                # haven't got enough waveforms, so go for the following chunk
                else:
                    subsample = subsample_seed
                    fReadSameChunkAgain = False
                    # fGoForAnotherChunk is True by default

                    aux = f"run_{run}_chunk_{current_chunk_iterator}_awpc_"+\
                        f"{round(len(aux_wfset.waveforms)/number_of_channels)}"
                    aux_saving_filename = f"{aux}_FS.pkl" if read_full_streaming_data else f"{aux}_ST.pkl"

                    if verbose:
                        print(
                            f"\t\t --> All of the waveforms from this chunk were read"
                        )
                        print(
                            "\t --> Saving them to a pickle file in "
                            f"{os.path.join(saving_folderpath, aux_saving_filename)} ..."
                        )

                    led_utils.dump_object_to_pickle(
                        aux_wfset,
                        saving_folderpath,
                        aux_saving_filename,
                        verbose=verbose
                    )

                    # Switch to next chunk if there's another chunk
                    current_chunk_iterator += 1
                    if current_chunk_iterator >= chunks_no:
                        if verbose:
                            print(
                                f"\t\t --> There are no more chunks available for this run. "
                                "Although the targeted number-of-waveforms per channel was "
                                "not achieved for this run, the available ones have been saved. "
                            )

                        fGoForAnotherChunk = False

                    else:
                        if verbose:
                            print(
                                f"\t\t --> Proceeding to look for "
                                f"{wvfs_left_to_read_for_this_run-len(aux_wfset.waveforms)} "
                                f"(={wvfs_left_to_read_for_this_run}-{len(aux_wfset.waveforms)}) "
                                f"waveforms from the following chunk ({current_chunk_iterator+1}/"
                                f"{chunks_no}) of this run ({run})."
                            )
                        
                    # But only read the waveforms that we need to add up to 
                    # average_wfs_per_channel * number_of_channels
                    wvfs_left_to_read_for_this_run -= len(aux_wfset.waveforms)
                    
            # In this case, WaveformSet_from_hdf5_file() is misbehaving
            else:
                raise Exception(
                    "In function save_run_to_pickled_WaveformSet(): "
                    f"WaveformSet_from_hdf5_file() is misbehaving. It read"
                    f" more waveforms ({len(aux_wfset.waveforms)}) than "
                    f"specified (wvfm_count={average_wfs_per_channel * number_of_channels})"
                )

# Parameter input

In [ ]:
# List of batches to target, p.e. [6, 7]
batch_nos = [6, 7]
for _ in batch_nos:
    if _ not in range(1, 8):
        raise Exception(
            f"Found batch number {_}. All the requested "
            "batch numbers must be in the range [1, 7]."
        )

# List of PDEs to target, p.e. [40, 45]
pdes = [45, 50]
for _ in pdes:
    if _ not in [40., 45., 50.]:
        raise Exception(
            f"Found PDE {_}. All requested PDEs must be "
            "one of the following values: [40., 45., 50.]"
        )

In [ ]:
# Path to the channels-per-run database of the LED-calibration (typically placed in src/waffles/np04_analysis/led_calibration/scripts/batch_pickle_generator.ipynb)
channels_per_run_database_filepath = ""
# The folder containing TXT files with the 0<run>.txt format with the needed rucio paths
base_rucio_filepaths_folderpath = ""
# The folder where the pickle'd WaveformSets will be saved
base_saving_folderpath = ""
# Average number of waveforms per channel
average_wfs_per_channel = 3000
# The first value to try for the 'subsample' parameter of WaveformSet_from_hdf5_file()
subsample_seed = 3
# Whether to print functioning related messages
verbose = True

# Retrieve data into pickles

In [ ]:
df = pd.read_csv(channels_per_run_database_filepath)

for batch_no in batch_nos:
    for pde in pdes:

        # Filter out the runs which do not belong to the current
        # batch and PDE
        filtered_df = df[
            (df['batch'] == batch_no) &
            (df['pde'] == pde/100.)
        ]

        # Tuple of run numbers to read for the current batch and PDE
        runs = tuple(filtered_df['run'].tolist())

        # acquired_apas[i] is the list of acquired-APAs numbers
        acquired_apas = tuple([
            led_utils.parse_numeric_list(x)
            for x in filtered_df['acquired_apas'].tolist()
        ])

        # The folder where to look for the rucio paths for this 
        # batch and PDE
        rucio_filepaths_folderpath_candidate = os.path.join(
            base_rucio_filepaths_folderpath,
            f"batch_{batch_no}/pde_{pde}"
        )

        if not os.path.exists(rucio_filepaths_folderpath_candidate):
            print(
                f"WARNING: The folder {rucio_filepaths_folderpath_candidate} "
                "does not exist. Skipping the data download for batch "
                f"{batch_no} and PDE {pde}."
            )
            continue

        for i, run in enumerate(runs):

            # Both flags may be simultaneously True, if a during a certain run
            # APA 1 and another APA were acquired at the same time. P.e. run 27898
            fReadFullStreamingData = False
            fReadSelfTriggerData = False

            if 1 in acquired_apas[i]:
                fReadFullStreamingData = True

            if 2 in acquired_apas[i] or 3 in acquired_apas[i] or 4 in acquired_apas[i]:
                fReadSelfTriggerData = True

            aux = df[
                df['run'] == run
            ].iloc[0]['aimed_channels']

            if aux == '[]':
                print(
                    f"WARNING: The run {run} does not have any "
                    "aimed channels defined in the database. Skipping "
                    f"this run."
                )
                continue

            # This is a dictionary of channels with the following format:
            # {112: [35, 37, 21, 23, 25], 113: [0], 111: [11, 44, 45]}
            formatted_channels = led_utils.arrange_dictionary_of_endpoints_and_channels(
                led_utils.parse_numeric_list(aux)
            )
            
            # The folder where the created pickles will be saved
            saving_folderpath = os.path.join(
                base_saving_folderpath,
                f"{average_wfs_per_channel}_wfs_per_channel"
            )

            if not os.path.exists(saving_folderpath):
                os.makedirs(saving_folderpath)

            # OPEN ISSUE: Doing several tests I noted that in the data-taking for APA 1
            # in batches 4 and 5, the WaveformSet_from_hdf5_file() function does not seem
            # to find data for the following channels {105: [0, 2, 3], 104: [10, 13, 14,
            # 15, 17], 107: [10, 12]} (it does find data for the channels {105: {24, 5},
            # 104: {16, 11, 12}}} though). The affected runs are 28634 (batch 4) and 28980
            # (batch 5). As a side effect, this code saves thrice as waveforms for the
            # found channels, which is not a big deal (it saves a fixed number of waveforms
            # among a number of channels which was expected to be thrice as big - 5 vs 15).
            # However, the reason why there is no data available for those channels in the
            # raw HDF5 files should be investigated.

            rucio_filepaths_folderpath=os.path.join(
                base_rucio_filepaths_folderpath,
                f"batch_{batch_no}/pde_{pde}"
            )

            if fReadFullStreamingData:
                save_run_to_pickled_WaveformSet(
                    run,
                    saving_folderpath,
                    average_wfs_per_channel=average_wfs_per_channel,
                    # For an acquired_apas=[1,2] case, we'll be giving APAs 1&2 channels
                    # in formatted_channels. Note, however, that
                    # reader.WaveformSet_from_hdf5_file will correctly identify which
                    # channels should be read thanks to the read_full_streaming_data
                    # parameter
                    channels=formatted_channels,
                    rucio_filepaths_folderpath=rucio_filepaths_folderpath,
                    read_full_streaming_data=True,
                    subsample_seed=subsample_seed,
                    verbose=verbose
                )

            if fReadSelfTriggerData:
                save_run_to_pickled_WaveformSet(
                    run,
                    saving_folderpath,
                    average_wfs_per_channel=average_wfs_per_channel,
                    channels=formatted_channels,
                    rucio_filepaths_folderpath=rucio_filepaths_folderpath,
                    read_full_streaming_data=False,
                    subsample_seed=subsample_seed,
                    verbose=verbose
                )

# Debugging

In [ ]:
# #### Code to test the number of retrieved waveforms, channel by channel
# from waffles.input_output.pickle_file_reader import WaveformSet_from_pickle_file
# from matplotlib import pyplot as plt

# # The following 5 parameters must be set mutually
# batch = 4
# pde = 45
# run = 28634
# read_full_streaming_data = True
# channels = led_utils.arrange_dictionary_of_endpoints_and_channels([10500, 10502, 10503, 10505, 10410, 10411, 10412, 10413, 10414, 10415, 10416, 10417, 10710, 10712, 10524])

# save_run_to_pickled_WaveformSet(
#     run,
#     "", # Path where to save the pickle
#     average_wfs_per_channel=200,
#     channels=channels,
#     rucio_filepaths_folderpath="", # Path where to look for the rucio paths
#     read_full_streaming_data=read_full_streaming_data,
#     subsample_seed=subsample_seed,
#     verbose=True
# )

# mywfset = WaveformSet_from_pickle_file(
#     "" # Copy here the output filepath prompted by the call to save_run_to_pickled_WaveformSet() above
# )

# print("---------------- Expected channels ----------------")
# print(channels)
# print("---------------- Found channels ----------------")
# print(mywfset.available_channels)

# print("---------------- number-of-waveorms distribution through channels ----------------")
# samples = []
# for endpoint in mywfset.available_channels[run].keys():
#     for channel in mywfset.available_channels[run][endpoint]:

#         count = 0
#         for wf in mywfset.waveforms:
#             if wf.endpoint == endpoint and wf.channel == channel:
#                 count += 1
#         print(f"Found {count} waveforms with endpoint {endpoint} and channel {channel} in the WaveformSet.")
#         samples.append(count)

# plt.hist(samples, bins=20)
# plt.show()